# Defined functions for Segmentation

In [4]:
import os
import cv2
import numpy as np
from PIL import Image

class yolo_txt:
    def __init__(self, image_path, yolo_label_path):
        self.image = cv2.imread(image_path)
        f = open(yolo_label_path, 'r')
        self.yolo_labels = f.readlines()
        self.label_name = yolo_label_path.split("/")[-1][:-4]
        self.image_name = yolo_label_path.split("/")[-1][:-4]

    def horizontal_flip(self):
        """ Function to flip the image horizontally along with yolo txt segmentations.

        Returns:
        - Horizontally flipped image
        - Horizontally flipped coordinates for each object label
        """
      
        image = self.image
        yolo_labels = self.yolo_labels

        flipped_image = cv2.flip(image, 1)
        flipped_labels = []
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            points = [float(x) for x in label_parts[1:]]
            img_height, img_width, _ = image.shape
            x_pts = []
            y_pts = []
            for j in range(len(points)):
                if j%2==0:
                    x_coord = points[j]
                    diff = 1 - x_coord
                    x_pts.append(diff)
                if j%2!=0:
                    y_pts.append(points[j])

            flipped_label = f"{class_id} "

            for k in range(len(x_pts)):
                if len(x_pts)==k+1:
                    l = f"{x_pts[k]} {y_pts[k]}\n"
                else:
                    l = f"{x_pts[k]} {y_pts[k]} "
                flipped_label+=l

            flipped_labels.append(flipped_label)
        
        return flipped_image, flipped_labels


    def vertical_flip(self):
        """ Function to flip the image vertically along with yolo txt segmentations.

        Returns:
        - Vertically flipped image
        - Vertically flipped coordinates for each object label
        """

        image = self.image
        yolo_labels = self.yolo_labels

        flipped_image = cv2.flip(image, 0)
        flipped_labels = []
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            points = [float(x) for x in label_parts[1:]]
            img_height, img_width, _ = image.shape
            x_pts = []
            y_pts = []
            for j in range(len(points)):
                if j%2==0:
                    x_pts.append(points[j])
                if j%2!=0:
                    y_coord = points[j]
                    diff = 1 - y_coord
                    y_pts.append(diff)
                    
            flipped_label = f"{class_id} "

            for k in range(len(x_pts)):
                if len(x_pts)==k+1:
                    l = f"{x_pts[k]} {y_pts[k]}\n"
                else:
                    l = f"{x_pts[k]} {y_pts[k]} "
                flipped_label+=l

            flipped_labels.append(flipped_label)
        
        return flipped_image, flipped_labels


    def save(self, image, label, output_dir_path, suffix):
        """ function to save images and corresponding yolo txt labels with the provided suffix name in a specified output directory.

        Parameters:
        - image: Image array
        - label: Txt label list
        - output_dir_path: Path of output directory
        - suffix: Preferred suffix name to file

        Returns:
        - Saves image and txt label file in a specified directory with specified suffix name.
        """

        crpt = []
        try:
            cv2.imwrite(output_dir_path + f"/{self.image_name}_{suffix}.jpg", image)
            for i in label:
                with open(output_dir_path + f"/{self.label_name}_{suffix}.txt", "a") as f:
                    f.writelines(i)
        except:
            crpt.append(self.image_name)
        return crpt

    # def direct_save(image_path, label_path, output_path, apply):
    #     """ function to save images and corresponding yolo txt labels after transformation with the transforming function name in a specified output directory """
    #     crpt = {"image_files":[], "label_files":[]}
    #     try:
    #         image = cv2.imread(image_path)
    #         f = open(label_path, 'r')
    #         yolo_labels = f.readlines()
    #         img, labels = apply(image, yolo_labels)
    #         cv2.imwrite(output_path + f"/{label_path.split('/')[-1][:-4]}_{apply.__name__}.jpg", img)
    #         for i in labels:
    #             with open(output_path + f"/{label_path.split('/')[-1][:-4]}_{apply.__name__}.txt", "w") as f:
    #                 f.writelines(i)
    #     except:
    #         crpt["image_files"].append(image_path)
    #         crpt["label_files"].append(label_path)

    #     return crpt
    

    def rotate_point(x, y, cx, cy, angle):
        """ 
        Function to rotate x, y co-ordinates based on angle and center of rotation.

        Parameters: 
        - x: Original x coordinate 
        - y: Original y coordinate
        - cx: Center of rotation x coordinate
        - cy: Center of rotation y coordinate

        Returns:
        - Rotated x co-ordinate
        - Rotated y co-ordinate
        """
        angle_rad = np.radians(angle)
        x_rotated = (x - cx) * np.cos(angle_rad) - (y - cy) * np.sin(angle_rad) + cx
        y_rotated = (x - cx) * np.sin(angle_rad) + (y - cy) * np.cos(angle_rad) + cy
        return x_rotated, y_rotated
    

    def add_padding_to_image(image, yolo_labels, padding_color=(0, 0, 0), padding_height_ratio=0.2, padding_width_ratio=0.2):
        """
        function to add padding to an image along with its coordinates for multiple objects.

        Parameters:
        - image: Input image array
        - object_coordinates: List of yolo txt label lines
        - padding_color: Color of the padding (default is black)
        - padding_ratio: Ratio of padding to the image dimensions

        Returns:
        - Padded image
        - Padded coordinates for each object label
        """

        height, width = image.shape[:2]
        padded_coordinates_dict = {}
        object_coordinates = {}
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            points = [float(x) for x in label_parts[1:]]
            x_pts = [points[i]*width for i in range(len(points)) if i%2==0]
            y_pts = [points[i]*height for i in range(len(points)) if i%2!=0]
            coordinates = []
            for y in range(len(x_pts)):
                coordinates.append((x_pts[y], y_pts[y]))
            object_coordinates[class_id] = coordinates

        # Calculate padding dimensions
        padding_height = int(height * padding_height_ratio)
        padding_width = int(width * padding_width_ratio)

        # Create a padded image
        padded_image = cv2.copyMakeBorder(image, padding_height, padding_height, padding_width, padding_width,
                                        cv2.BORDER_CONSTANT, value=padding_color)
        padded_img_height, padded_img_width = padded_image.shape[:2]
        # Update coordinates based on padding
        for label, coordinates in object_coordinates.items():
            # Add padding to the coordinates
            padded_coordinates = [(x + padding_width, y + padding_height) for x, y in coordinates]
            padded_coordinates_dict[label] = padded_coordinates
        txt_labels = []
        for label_id, coords in padded_coordinates_dict.items():
            l = f"{label_id} "
            for k in range(len(coords)):
                if len(coords)==k+1:
                    x_point = coords[k][0]
                    y_point = coords[k][1]
                    l1 = f"{x_point/padded_img_width} {y_point/padded_img_height}\n"
                else:
                    x_point = coords[k][0]
                    y_point = coords[k][1]
                    l1 = f"{x_point/padded_img_width} {y_point/padded_img_height} "
                l+=l1
            txt_labels.append(l)
            
        return padded_image, txt_labels


    def calculate_padding_ratio(image, yolo_labels, angle):
        """
        Calculate the padding ratio based on the bounding box dimensions of biggest segmented object in an image after rotation.
        
        Parameters:
        - image: Input image
        - object_coordinates: List of (x, y) coordinates
        - angle: Rotation angle in degrees

        Returns:
        - Padding ratio
        """
        height, width = image.shape[:2]
        object_coordinates = {}
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            points = [float(x) for x in label_parts[1:]]
            x_pts = [points[i]*width for i in range(len(points)) if i%2==0]
            y_pts = [points[i]*height for i in range(len(points)) if i%2!=0]
            coordinates = []
            for y in range(len(x_pts)):
                coordinates.append((x_pts[y], y_pts[y]))
            object_coordinates[class_id] = coordinates

        rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
        rotated_heights = []
        rotated_widths = []
        for label_id, coords in object_coordinates.items():
            rotated_coordinates = cv2.transform(np.array([coords]), rotation_matrix)[0]
            min_x, min_y = np.min(rotated_coordinates, axis=0)
            max_x, max_y = np.max(rotated_coordinates, axis=0)
            rotated_width = max_x - min_x
            rotated_height = max_y - min_y
            rotated_heights.append(rotated_height)
            rotated_widths.append(rotated_width)

        padding_ratio_width = (max(rotated_widths)-width) / width
        padding_ratio_height = (max(rotated_heights)-height) / height

        return abs(padding_ratio_width), abs(padding_ratio_height)


    def calculate_padding_ratios(W, H, theta):
        """
        Calculate the padding ratio based on the bounding box dimensions of image after rotation.

        Parameters:
        - W: Original width
        - H: Original height
        - theta: Rotation angle in degrees

        Returns:
        - Padding ratio
        """
        W_prime = W * np.cos(theta) + H * np.sin(theta)
        H_prime = W * np.sin(theta) + H * np.cos(theta)
        max_displacement_x = (W_prime - W) / 2
        max_displacement_y = (H_prime - H) / 2
        pad_w_ratio = max_displacement_x / W
        pad_h_ratio = max_displacement_y / H

        return abs(pad_w_ratio), abs(pad_h_ratio)


    def rotate(self, angle=30):

        """ Function to rotate image and corresponding masks.
        
        Parameters:
        - angle: Rotation angle in degree

        Returns:
        - Rotated image array
        - Rotated coordinates for each object label
        """

        image = self.image
        yolo_labels = self.yolo_labels
        padding_width_ratio, padding_height_ratio = yolo_txt.calculate_padding_ratios(image.shape[1], image.shape[0], angle)
        image, yolo_labels = yolo_txt.add_padding_to_image(image, yolo_labels, max(padding_height_ratio, padding_width_ratio), max(padding_height_ratio, padding_width_ratio))
        # rotating image
        height, width, _ = image.shape
        rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

       
        # rotating labels
        rotated_labels = []
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            points = [float(x) for x in label_parts[1:]]
            x_pts = [points[i] for i in range(len(points)) if i%2==0]
            y_pts = [points[i] for i in range(len(points)) if i%2!=0]
            rotated_x_points = []
            rotated_y_points = []
            for j in range(len(x_pts)):
                x = x_pts[j] * width
                y = y_pts[j] * height
                x_rotated, y_rotated = yolo_txt.rotate_point(x, y, width/2, height/2, -angle)
                rotated_x_points.append(x_rotated/width)
                rotated_y_points.append(y_rotated/height)
            
            rotated_label = f"{class_id} "
            for k in range(len(rotated_x_points)):
                if len(rotated_x_points)==k+1:
                    x_point = max(0, min(1, rotated_x_points[k]))
                    y_point = max(0, min(1, rotated_y_points[k])) 
                    # x_point = rotated_x_points[k]
                    # y_point = rotated_y_points[k]
                    l = f"{x_point} {y_point}\n"
                else:
                    x_point = max(0, min(1, rotated_x_points[k]))
                    y_point = max(0, min(1, rotated_y_points[k]))
                    # x_point = rotated_x_points[k]
                    # y_point = rotated_y_points[k] 
                    l = f"{x_point} {y_point} "
                rotated_label+=l

            rotated_labels.append(rotated_label)

        return rotated_image, rotated_labels


    def hsv_transform(self, brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5):
        """ 
        Function to apply color transformations on an image. It will pass anotations as it is.

        Parameters:     
        1. Brightness: Common range: -0.5 to 0.5
            Values closer to 0 maintain the original brightness, while negative values darken the image, and positive values brighten it.
        2. Contrast: Common range: 0.2 to 2.0
            Values closer to 1.0 maintain the original contrast, while values below 1.0 decrease contrast, and values above 1.0 increase it.
        3. Saturation: Common range: 0.2 to 2.0
            Values closer to 1.0 maintain the original saturation, while values below 1.0 desaturate the image, and values above 1.0 saturate it.
        4. Hue: Common range: -0.5 to 0.5
            Values closer to 0.0 maintain the original hue, while negative and positive values shift the hue. 

        Returns:
        - Image array
        - Yolo label list
        """
        
        image = self.image
        yolo_labels = self.yolo_labels

        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # brightness
        hsv_image[:, :, 2] = np.clip(hsv_image[:, :, 2] * (1 + brightness), 0, 255)
        # contrast
        hsv_image[:, :, 2] = np.clip((hsv_image[:, :, 2] - 128) * (1 + contrast) + 128, 0, 255)
        # saturation
        hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] * (1 + saturation), 0, 255)
        # hue
        hsv_image[:, :, 0] = (hsv_image[:, :, 0] + hue * 360) % 180

        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

        return augmented_image, yolo_labels
    
    def is_point_inside_box(x, y, xmin, ymin, xmax, ymax):
        """
        Function to check whether the point lies in bounding box or not.

        Parameters:
        - x: x coordinate of point
        - y: y coordinate of point
        - xmin: xmin coordinate of bounding box to check
        - ymin: ymin coordinate of bounding box to check
        - xmax: xmax coordinate of bounding box to check
        - ymax: ymax coordinate of bounding box to check

        Returns: 
        - True if points lies in bounding box
        - False if points does not lie in bounding box
        """
        return xmin <= x <= xmax and ymin <= y <= ymax
    

    def is_mask_overlapping_bbox(mask_coordinates, xmin, ymin, xmax, ymax):
        """
        Function to check whether the mask lies in bounding box or not.

        Parameters:
        - mask_coordinates: List of coordinate in a tuple i.e. [(x,y), ..., (xn, yn)]
        - xmin: xmin coordinate of bounding box to check
        - ymin: ymin coordinate of bounding box to check
        - xmax: xmax coordinate of bounding box to check
        - ymax: ymax coordinate of bounding box to check

        Returns: 
        - True if points lies in bounding box
        - False if points does not lie in bounding box
        """

        for x, y in mask_coordinates:
            if xmin <= x <= xmax and ymin <= y <= ymax:
                return True
        return False
    

    def is_bbox_overlapping_mask(xmin, ymin, xmax, ymax, mask_coordinates):
        """
        Function to check whether the bounding box is completely inside a mask or not.

        Parameters:
        - xmin: xmin coordinate of bounding box to check
        - ymin: ymin coordinate of bounding box to check
        - xmax: xmax coordinate of bounding box to check
        - ymax: ymax coordinate of bounding box to check
         - mask_coordinates: List of coordinate in a tuple i.e. [(x,y), ..., (xn, yn)]

        Returns: 
        - True if bounding box points lies in mask
        - False if bounding box points does not lie in the mask
        """
        bbox_coordinates = xmin, ymin, xmax, ymax
        # Convert the bounding box coordinates to a rectangle
        bbox_rect = np.array(bbox_coordinates).reshape((-1, 2))
        # Convert the mask coordinates to a polygon
        mask_polygon = np.array(mask_coordinates, dtype=np.float32)
        # Check if all points of the rectangle are inside the polygon
        return all(cv2.pointPolygonTest(mask_polygon, (x, y), False) >= 0 for x, y in bbox_rect)


    def zoom(self, zoom_factor=2.0):
        """ Function to zoom image and corresponding masks.
        
        Parameters:
        - zoom_factor: Degree of zoom. (determines the degree of zoom. A factor greater than 1.0 enlarges the image. A factor between 0 to 0.3 shrinks the image. and Default is 2x)

        Returns:
        - zoomed image array
        - zoomed coordinates for each object label 
        """ 
        if zoom_factor<1 and zoom_factor>0.34:
            zoom_factor = 2.0
            
        image = self.image
        yolo_labels = self.yolo_labels

        # Zooming image
        height, width, channels = image.shape
        new_width = int(width * zoom_factor)
        new_height = int(height * zoom_factor)
        zoomed_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        crop_left = (new_width - width) // 2
        crop_top = (new_height - height) // 2
        crop_right = crop_left + width
        crop_bottom = crop_top + height
        cropped_zoomed_image = zoomed_image[crop_top:crop_bottom, crop_left:crop_right]
        xmin = crop_left/zoom_factor
        xmax = crop_right/zoom_factor
        ymin = crop_top/zoom_factor
        ymax = crop_bottom/zoom_factor

        # Zooming labels
        zoomed_height, zoomed_width, _ = cropped_zoomed_image.shape
        zoomed_yolo_labels = []
        object_coordinates = {}
        scaled_labels = {}
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            points = [float(x) for x in label_parts[1:]]
            x_pts = [points[i]*width for i in range(len(points)) if i%2==0]
            y_pts = [points[i]*height for i in range(len(points)) if i%2!=0]
            coordinates = []
            for y in range(len(x_pts)):
                coordinates.append((x_pts[y], y_pts[y]))
            object_coordinates[class_id] = coordinates

        c1 = set()
        for label_id, coords in object_coordinates.items():
                    if yolo_txt.is_mask_overlapping_bbox(coords, xmin, ymin, xmax, ymax):
                        c1.add(label_id)
                    if yolo_txt.is_bbox_overlapping_mask(xmin, ymin, xmax, ymax, coords):
                        c1.add(label_id)
        if len(c1)>0:
            for label_id, coords in object_coordinates.items():
                    coordinates = []
                    if zoom_factor>1:
                        for c in range(len(coords)):
                            x = coords[c][0]
                            y = coords[c][1]
                            scaled_x = int((x * zoom_factor) - (width * (zoom_factor - 1) / 2))
                            scaled_y = int((y * zoom_factor) - (height * (zoom_factor - 1) / 2))
                            coordinates.append((scaled_x, scaled_y))
                        scaled_labels[label_id] = coordinates
                    if zoom_factor<1:
                        for c in range(len(coords)):
                            x = coords[c][0]
                            y = coords[c][1]
                            scaled_x = int(x * zoom_factor)
                            scaled_y = int(y * zoom_factor)
                            coordinates.append((scaled_x, scaled_y))
                        scaled_labels[label_id] = coordinates

            for label_id, coords in scaled_labels.items():
                if label_id in c1:
                    l = f"{label_id} "
                    for k in range(len(coords)):
                        if len(coords)==k+1:
                            x_point = max(0, min(1, coords[k][0]/zoomed_width))
                            y_point = max(0, min(1, coords[k][1]/zoomed_height)) 
                            l1 = f"{x_point} {y_point}\n"
                        else:
                            x_point = max(0, min(1, coords[k][0]/zoomed_width))
                            y_point = max(0, min(1, coords[k][1]/zoomed_height))
                            l1 = f"{x_point} {y_point} "
                        l+=l1
                    zoomed_yolo_labels.append(l)

        return cropped_zoomed_image, zoomed_yolo_labels


    def rotate_and_zoom(self, angle=30, zoom_factor=5.0):
        """ Function to zoom image and corresponding masks.
        
        Parameters:
        - zoom_factor: Degree of zoom. (determines the degree of zoom. A factor greater than 1.0 enlarges the image. A factor between 0 to 0.3 shrinks the image. and Default is 2x)

        Returns:
        - zoomed image array
        - zoomed coordinates for each object label 
        """ 

        image, yolo_labels = yolo_txt.rotate(self, angle)

        if zoom_factor<1 and zoom_factor>0.34:
            zoom_factor = 2.0

        # Zooming image
        height, width, channels = image.shape
        new_width = int(width * zoom_factor)
        new_height = int(height * zoom_factor)
        zoomed_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        crop_left = (new_width - width) // 2
        crop_top = (new_height - height) // 2
        crop_right = crop_left + width
        crop_bottom = crop_top + height
        rotated_zoomed_image = zoomed_image[crop_top:crop_bottom, crop_left:crop_right]
        xmin = crop_left/zoom_factor
        xmax = crop_right/zoom_factor
        ymin = crop_top/zoom_factor
        ymax = crop_bottom/zoom_factor

        # Zooming labels
        zoomed_height, zoomed_width, _ = rotated_zoomed_image.shape
        rotated_yolo_labels = []
        object_coordinates = {}
        scaled_labels = {}
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            points = [float(x) for x in label_parts[1:]]
            x_pts = [points[i]*width for i in range(len(points)) if i%2==0]
            y_pts = [points[i]*height for i in range(len(points)) if i%2!=0]
            coordinates = []
            for y in range(len(x_pts)):
                coordinates.append((x_pts[y], y_pts[y]))
            object_coordinates[class_id] = coordinates

        c1 = set()
        for label_id, coords in object_coordinates.items():
                    if yolo_txt.is_mask_overlapping_bbox(coords, xmin, ymin, xmax, ymax):
                        c1.add(label_id)
                    if yolo_txt.is_bbox_overlapping_mask(xmin, ymin, xmax, ymax, coords):
                        c1.add(label_id)
        if len(c1)>0:
            for label_id, coords in object_coordinates.items():
                    coordinates = []
                    if zoom_factor>1:
                        for c in range(len(coords)):
                            x = coords[c][0]
                            y = coords[c][1]
                            scaled_x = int((x * zoom_factor) - (width * (zoom_factor - 1) / 2))
                            scaled_y = int((y * zoom_factor) - (height * (zoom_factor - 1) / 2))
                            coordinates.append((scaled_x, scaled_y))
                        scaled_labels[label_id] = coordinates
                    if zoom_factor<1:
                        for c in range(len(coords)):
                            x = coords[c][0]
                            y = coords[c][1]
                            scaled_x = int(x * zoom_factor)
                            scaled_y = int(y * zoom_factor)
                            coordinates.append((scaled_x, scaled_y))
                        scaled_labels[label_id] = coordinates

            for label_id, coords in scaled_labels.items():
                if label_id in c1:
                    l = f"{label_id} "
                    for k in range(len(coords)):
                        if len(coords)==k+1:
                            x_point = max(0, min(1, coords[k][0]/zoomed_width))
                            y_point = max(0, min(1, coords[k][1]/zoomed_height)) 
                            l1 = f"{x_point} {y_point}\n"
                        else:
                            x_point = max(0, min(1, coords[k][0]/zoomed_width))
                            y_point = max(0, min(1, coords[k][1]/zoomed_height))
                            l1 = f"{x_point} {y_point} "
                        l+=l1
                    rotated_yolo_labels.append(l)
        return rotated_zoomed_image, rotated_yolo_labels
    
        
    def brightness_transform(self, brightness=0.2):
        """ 
        Function to apply brightness transformations on an image. It will pass anotations as it is. Common range: -0.5 to 0.5
        Returns:
        - Image array
        - Yolo label list
        """
        image = self.image
        yolo_labels = self.yolo_labels
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image[:, :, 2] = np.clip(hsv_image[:, :, 2] * (1 + brightness), 0, 255)
        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        return augmented_image, yolo_labels

    def contrast_transform(self, contrast=1.15):
        """ 
        Function to apply contrast transformations on an image. It will pass anotations as it is. Common range: 0.2 to 2.0
        Returns:
        - Image array
        - Yolo label list
        """
        image = self.image
        yolo_labels = self.yolo_labels
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image[:, :, 2] = np.clip((hsv_image[:, :, 2] - 128) * (1 + contrast) + 128, 0, 255)
        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        return augmented_image, yolo_labels

    def saturation_transform(self, saturation=1.15):
        """ 
        Function to apply saturation transformations on an image. It will pass anotations as it is. Common range: 0.2 to 2.0
        Returns:
        - Image array
        - Yolo label list
        """
        image = self.image
        yolo_labels = self.yolo_labels
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] * (1 + saturation), 0, 255)
        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        return augmented_image, yolo_labels

    def hue_transform(self, hue=0.2):
        """ 
        Function to apply hue transformations on an image. It will pass anotations as it is. Common range: -0.5 to 0.5
        Returns:
        - Image array
        - Yolo
        """
        image = self.image
        yolo_labels = self.yolo_labels
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image[:, :, 0] = (hsv_image[:, :, 0] + hue * 360) % 180
        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        return augmented_image, yolo_labels


## Refer this code to create augmentations for entire folder containing images and txt labels - 

In [5]:
from tqdm import tqdm
from random import seed
from random import choice

input_path = "/input/path/to/data"
output_dir  = "/save/directory/path/"


seed(1)
brightness_sequence = [-0.3, -0.2, 0.2, 0.3]
saturation_sequence = [0.7, 1.2, 1.4, 1.6]
contrast_sequence = [0.7, 1.2, 1.4, 1.6]
hue_sequence = [-0.3, -0.2, 0.2, 0.3]

for i in os.listdir(input_path):
    if i.endswith(".jpg"):
        obj = yolo_txt(input_path + "/" + i, input_path + "/" + i[:-3]+"txt")
        l = yolo_txt.rotate(obj, 90)[1]
        yolo_txt.save(obj, yolo_txt.horizontal_flip(obj)[0], yolo_txt.horizontal_flip(obj)[1], output_dir, "hflip")
        yolo_txt.save(obj, yolo_txt.vertical_flip(obj)[0], yolo_txt.vertical_flip(obj)[1], output_dir, "vflip")
        yolo_txt.save(obj, yolo_txt.hsv_transform(obj, choice(brightness_sequence), choice(contrast_sequence), choice(saturation_sequence), choice(hue_sequence))[0], yolo_txt.hsv_transform(obj, choice(brightness_sequence), choice(contrast_sequence), choice(saturation_sequence), choice(hue_sequence))[1], output_dir, "hsv_0")
        yolo_txt.save(obj, yolo_txt.hsv_transform(obj, choice(brightness_sequence), choice(contrast_sequence), choice(saturation_sequence), choice(hue_sequence))[0], yolo_txt.hsv_transform(obj, choice(brightness_sequence), choice(contrast_sequence), choice(saturation_sequence), choice(hue_sequence))[1], output_dir, "hsv_1")
        yolo_txt.save(obj, yolo_txt.hsv_transform(obj, choice(brightness_sequence), choice(contrast_sequence), choice(saturation_sequence), choice(hue_sequence))[0], yolo_txt.hsv_transform(obj, choice(brightness_sequence), choice(contrast_sequence), choice(saturation_sequence), choice(hue_sequence))[1], output_dir, "hsv_2")
        yolo_txt.save(obj, yolo_txt.rotate(obj, 30)[0], yolo_txt.rotate(obj, 30)[1], output_dir, "rotate_30")
        yolo_txt.save(obj, yolo_txt.rotate(obj, 70)[0], yolo_txt.rotate(obj, 70)[1], output_dir, "rotate_70")
        yolo_txt.save(obj, yolo_txt.zoom(obj, 1.5)[0], yolo_txt.zoom(obj, 1.5)[1], output_dir, "zoom_1.5")

# Defined functions for Detection

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image

class yolo_txt:
    def __init__(self, image_path, yolo_label_path):
        self.image = cv2.imread(image_path)
        f = open(yolo_label_path, 'r')
        self.yolo_labels = f.readlines()
        self.label_name = yolo_label_path.split("/")[-1][:-4]
        self.image_name = yolo_label_path.split("/")[-1][:-4]


    def horizontal_flip(self):
        """ Function to flip the image horizontally along with yolo txt detection boxes.

        Returns:
        - Horizontally flipped image
        - Horizontally flipped coordinates for each object label
        """
      
        image = self.image
        yolo_labels = self.yolo_labels

        flipped_image = cv2.flip(image, 1)
        flipped_labels = []
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            x_pt = float(label_parts[1])
            y_pt = float(label_parts[2])
            x_W = float(label_parts[3])
            y_h = float(label_parts[4])
            x_diff = 1 - x_pt
            x_point = max(0, min(1, x_diff))
            y_point = max(0, min(1, y_pt)) 
            l = f"{class_id} {x_point} {y_point} {x_W} {y_h}\n"
            flipped_labels.append(l)
        
        return flipped_image, flipped_labels



    def vertical_flip(self):
        """ Function to flip the image vertically along with yolo txt detection boxes.

        Returns:
        - Vertically flipped image
        - Vertically flipped coordinates for each object label
        """

        image = self.image
        yolo_labels = self.yolo_labels

        flipped_image = cv2.flip(image, 0)
        flipped_labels = []
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            x_pt = float(label_parts[1])
            y_pt = float(label_parts[2])
            x_W = float(label_parts[3])
            y_h = float(label_parts[4])
            y_diff = 1 - y_pt
            x_point = max(0, min(1, x_pt))
            y_point = max(0, min(1, y_diff)) 
            l = f"{class_id} {x_point} {y_point} {x_W} {y_h}\n"
            flipped_labels.append(l)
        
        return flipped_image, flipped_labels


    def save(self, image, label, output_dir_path, suffix):
        """ function to save images and corresponding yolo txt labels with the provided suffix name in a specified output directory.

        Parameters:
        - image: Image array
        - label: Txt label list
        - output_dir_path: Path of output directory
        - suffix: Preferred suffix name to file

        Returns:
        - Saves image and txt label file in a specified directory with specified suffix name.
        """

        crpt = []
        try:
            cv2.imwrite(output_dir_path + f"/{self.image_name}_{suffix}.jpg", image)
            for i in label:
                with open(output_dir_path + f"/{self.label_name}_{suffix}.txt", "a") as f:
                    f.writelines(i)
        except:
            crpt.append(self.image_name)
        return crpt


    def brightness_transform(self, brightness=0.2):
        """ 
        Function to apply brightness transformations on an image. It will pass anotations as it is. Common range: -0.5 to 0.5
        Returns:
        - Image array
        - Yolo label list
        """
        image = self.image
        yolo_labels = self.yolo_labels
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image[:, :, 2] = np.clip(hsv_image[:, :, 2] * (1 + brightness), 0, 255)
        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        return augmented_image, yolo_labels

    def contrast_transform(self, contrast=1.15):
        """ 
        Function to apply contrast transformations on an image. It will pass anotations as it is. Common range: 0.2 to 2.0
        Returns:
        - Image array
        - Yolo label list
        """
        image = self.image
        yolo_labels = self.yolo_labels
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image[:, :, 2] = np.clip((hsv_image[:, :, 2] - 128) * (1 + contrast) + 128, 0, 255)
        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        return augmented_image, yolo_labels

    def saturation_transform(self, saturation=1.15):
        """ 
        Function to apply saturation transformations on an image. It will pass anotations as it is. Common range: 0.2 to 2.0
        Returns:
        - Image array
        - Yolo label list
        """
        image = self.image
        yolo_labels = self.yolo_labels
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] * (1 + saturation), 0, 255)
        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        return augmented_image, yolo_labels

    def hue_transform(self, hue=0.2):
        """ 
        Function to apply hue transformations on an image. It will pass anotations as it is. Common range: -0.5 to 0.5
        Returns:
        - Image array
        - Yolo
        """
        image = self.image
        yolo_labels = self.yolo_labels
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image[:, :, 0] = (hsv_image[:, :, 0] + hue * 360) % 180
        augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        return augmented_image, yolo_labels


    def erode(self, kernel=np.ones((3, 3), np.uint8)):
        image = self.image
        yolo_labels = self.yolo_labels
        eroded_image = cv2.erode(image, kernel, iterations=1)
        return eroded_image, yolo_labels


    def dilate(self, kernel=np.ones((3, 3), np.uint8)):
        image = self.image
        yolo_labels = self.yolo_labels
        dilated_image = cv2.dilate(image, kernel, iterations=1)
        return dilated_image, yolo_labels


    def zoom(self, zoom_factor=1.2):
        """ 
        Function to zoom into the image along with yolo txt detection boxes.
        Parameters:
        - zoom_factor: Factor by which to zoom into the image
        Returns:
        - Zoomed image
        - Updated yolo labels for the zoomed image
        """
        image = self.image
        yolo_labels = self.yolo_labels

        height, width = image.shape[:2]
        new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)

        resized_image = cv2.resize(image, (new_width, new_height))
        start_x = (new_width - width) // 2
        start_y = (new_height - height) // 2

        cropped_image = resized_image[start_y:start_y + height, start_x:start_x + width]

        zoomed_labels = []
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            x_pt = float(label_parts[1])
            y_pt = float(label_parts[2])
            x_W = float(label_parts[3])
            y_h = float(label_parts[4])

            x_center = x_pt * width * zoom_factor
            y_center = y_pt * height * zoom_factor

            x_point = (x_center - start_x) / width
            y_point = (y_center - start_y) / height

            if 0 <= x_point <= 1 and 0 <= y_point <= 1:
                l = f"{class_id} {x_point} {y_point} {x_W} {y_h}\n"
                zoomed_labels.append(l)

        return cropped_image, zoomed_labels



    def rotate(self, angle=15, center=None):
        image = self.image
        yolo_labels = self.yolo_labels

        height, width = image.shape[:2]

        if center is None:
            center = (width // 2, height // 2)

        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        cos = np.abs(rotation_matrix[0, 0])
        sin = np.abs(rotation_matrix[0, 1])

        new_width = int((height * sin) + (width * cos))
        new_height = int((height * cos) + (width * sin))

        rotation_matrix[0, 2] += (new_width / 2) - center[0]
        rotation_matrix[1, 2] += (new_height / 2) - center[1]

        rotated_image = cv2.warpAffine(image, rotation_matrix, (new_width, new_height))

        rotated_labels = []
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            x_pt = float(label_parts[1]) * width
            y_pt = float(label_parts[2]) * height
            x_W = float(label_parts[3])
            y_h = float(label_parts[4])

            points = np.array([[x_pt, y_pt]])
            new_points = cv2.transform(np.array([points]), rotation_matrix)[0]

            x_point = new_points[0][0] / new_width
            y_point = new_points[0][1] / new_height

            if 0 <= x_point <= 1 and 0 <= y_point <= 1:
                l = f"{class_id} {x_point} {y_point} {x_W} {y_h}\n"
                rotated_labels.append(l)

        return rotated_image, rotated_labels



    def rotate_and_zoom(self, angle=15, zoom_factor=1.2, center=None):
        """
        Function to rotate and zoom the image along with yolo txt detection boxes.
        Parameters:
        - angle: Degree by which to rotate the image
        - zoom_factor: Factor by which to zoom into the image
        - center: Center point for rotation
        Returns:
        - Rotated and zoomed image
        - Updated yolo labels for the rotated and zoomed image
        """
        image = self.image
        yolo_labels = self.yolo_labels

        height, width = image.shape[:2]

        # Zoom
        new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
        resized_image = cv2.resize(image, (new_width, new_height))
        start_x = (new_width - width) // 2
        start_y = (new_height - height) // 2
        zoomed_image = resized_image[start_y:start_y + height, start_x:start_x + width]

        zoomed_labels = []
        for label in yolo_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            x_pt = float(label_parts[1])
            y_pt = float(label_parts[2])
            x_W = float(label_parts[3])
            y_h = float(label_parts[4])

            x_center = x_pt * width * zoom_factor - start_x
            y_center = y_pt * height * zoom_factor - start_y

            if 0 <= x_center <= width and 0 <= y_center <= height:
                zoomed_labels.append(f"{class_id} {x_center / width} {y_center / height} {x_W} {y_h}\n")

        # Rotate
        if center is None:
            center = (width // 2, height // 2)

        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        cos = np.abs(rotation_matrix[0, 0])
        sin = np.abs(rotation_matrix[0, 1])

        new_width = int((height * sin) + (width * cos))
        new_height = int((height * cos) + (width * sin))

        rotation_matrix[0, 2] += (new_width / 2) - center[0]
        rotation_matrix[1, 2] += (new_height / 2) - center[1]

        rotated_image = cv2.warpAffine(zoomed_image, rotation_matrix, (new_width, new_height))

        rotated_labels = []
        for label in zoomed_labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])
            x_pt = float(label_parts[1]) * width
            y_pt = float(label_parts[2]) * height
            x_W = float(label_parts[3])
            y_h = float(label_parts[4])

            points = np.array([[x_pt, y_pt]])
            new_points = cv2.transform(np.array([points]), rotation_matrix)[0]

            x_point = new_points[0][0] / new_width
            y_point = new_points[0][1] / new_height

            if 0 <= x_point <= 1 and 0 <= y_point <= 1:
                rotated_labels.append(f"{class_id} {x_point} {y_point} {x_W} {y_h}\n")

        return rotated_image, rotated_labels

## Refer this code to create augmentations for entire folder containing images and txt labels - 

In [ ]:
from tqdm import tqdm
from random import seed
from random import choice

input_path = "/input/path/to/data"
output_dir  = "/save/directory/path/"

seed(1)
brightness_sequence = [-0.2, -0.1, 0.1, 0.2]
saturation_sequence = [0.89, 0.94, 1.07, 1.3]
contrast_sequence = [0.99, 1.01, 1.02, 1.03]
hue_sequence = [-0.2, -0.1, 0.1, 0.2]


for i in tqdm(os.listdir(input_path)):
    if i.endswith(".jpg"):
        try:
            obj = yolo_txt(input_path + "/" + i, input_path + "/" + i[:-3]+"txt")
            yolo_txt.save(obj, yolo_txt.horizontal_flip(obj)[0], yolo_txt.horizontal_flip(obj)[1], output_dir, "hflip")
            yolo_txt.save(obj, yolo_txt.vertical_flip(obj)[0], yolo_txt.vertical_flip(obj)[1], output_dir, "vflip")
            yolo_txt.save(obj, yolo_txt.vertical_flip(obj)[0], yolo_txt.vertical_flip(obj)[1], output_dir, "vflip")

            brightness = choice(brightness_sequence)
            yolo_txt.save(obj, yolo_txt.brightness_transform(obj, brightness)[0], yolo_txt.brightness_transform(obj, brightness)[1], output_dir, f"brightness_{brightness}")
            saturation = choice(saturation_sequence)
            yolo_txt.save(obj, yolo_txt.saturation_transform(obj, saturation)[0], yolo_txt.saturation_transform(obj, saturation)[1], output_dir, f"saturation_{saturation}")
            hue = choice(hue_sequence)
            yolo_txt.save(obj, yolo_txt.hue_transform(obj, hue)[0], yolo_txt.hue_transform(obj, hue)[1], output_dir, f"hue_{hue}")

            # Erosion
            yolo_txt.save(obj, yolo_txt.erode(obj)[0], yolo_txt.erode(obj)[1], output_dir, "eroded")
            # Dilation
            yolo_txt.save(obj, yolo_txt.dilate(obj)[0], yolo_txt.dilate(obj)[1], output_dir, "dilated")
            angle = 30
            yolo_txt.save(obj, yolo_txt.rotate(obj, angle)[0], yolo_txt.rotate(obj, angle)[1], output_dir, f"rotate_{angle}")
            angle = 50
            yolo_txt.save(obj, yolo_txt.rotate(obj, angle)[0], yolo_txt.rotate(obj, angle)[1], output_dir, f"rotate_{angle}")
            angle = 80
            yolo_txt.save(obj, yolo_txt.rotate(obj, angle)[0], yolo_txt.rotate(obj, angle)[1], output_dir, f"rotate_{angle}")
            zoom_scale = 1.3
            yolo_txt.save(obj, yolo_txt.zoom(obj, zoom_scale)[0], yolo_txt.zoom(obj, zoom_scale)[1], output_dir, f"zoom_{zoom_scale}")
            zoom_scale = 1.6
            yolo_txt.save(obj, yolo_txt.zoom(obj, zoom_scale)[0], yolo_txt.zoom(obj, zoom_scale)[1], output_dir, f"zoom_{zoom_scale}")
            zoom_scale = 2.1
            yolo_txt.save(obj, yolo_txt.zoom(obj, zoom_scale)[0], yolo_txt.zoom(obj, zoom_scale)[1], output_dir, f"zoom_{zoom_scale}")
            angle, zoom_scale = 70, 1.7
            yolo_txt.save(obj, yolo_txt.rotate_and_zoom(obj, angle, zoom_scale)[0], yolo_txt.rotate_and_zoom(obj, angle, zoom_scale)[1], output_dir, f"rotate_{angle}_and_zoom_{zoom_scale}")
            angle, zoom_scale = 45, 2
            yolo_txt.save(obj, yolo_txt.rotate_and_zoom(obj, angle, zoom_scale)[0], yolo_txt.rotate_and_zoom(obj, angle, zoom_scale)[1], output_dir, f"rotate_{angle}_and_zoom_{zoom_scale}")
        
        except:
            continue